The workflow of our model(content_base) will be as follows:

1.Read bookid,userid,rating(rating datasets)

2.Read bookid and metadata info(metadata datasets)

3.Build SVD model for rating prediction. it takes  (userid,bookid)===>>> est:rating prediction

4.Make a generate_recommendation function(user_id, model, metadata, thresh=4)===>>select a title from all titles then call predict_review to find id of nearest title of book and call get_book_info to show metadata of selected book

5.Make predict_review function(user_id, book_title, model, metadata)==>>call get_book_id then use svd model to predict rating(user_id,book_id) and return book_id
    
6.Make a get_book_id function(book_title,metadata)==>>take book_title then find(difflib) and return book_id of nearest title 

7-Make get_book_info(book_id, metadata):it takes book_id and return metadata of this book

In [2]:
# install surprise library
#pip install scikit-surprise

  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-win_amd64.whl size=720941 sha256=a89a4d00c3ff1bb84280a6aac141bcab5b69566bdc8146744621ab12705c13fc
  Stored in directory: c:\users\admin\appdata\local\pip\cache\wheels\76\44\74\b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd

### Reading the Data

How to read  file from github:

You can copy/paste the url and change 2 things:

--Remove "blob"

--Replace github.com by raw.githubusercontent.com

In [2]:
ratings_data = pd.read_csv('https://raw.githubusercontent.com/AmolMavuduru/SurprisePythonExamples/master/data/ratings.csv.zip')
ratings_data.head(10)

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
5,1,2077,4
6,1,2487,4
7,1,2900,5
8,1,3662,4
9,1,3922,5


In [46]:
# The books that userid=1000 rates
ratings_data.loc[ratings_data['user_id'] == 314]

,book_id,user_id,rating
0,1,314,5
200,3,314,3
400,5,314,4
501,6,314,5
1100,12,314,4
...,...,...,...
677901,6815,314,3
699049,7033,314,4
724470,7293,314,4
776749,7830,314,3


In [47]:
books_metadata.loc[books_metadata['book_id'] == 3]['title']

1    Harry Potter and the Sorcerer's Stone (Harry P...
Name: title, dtype: object

In [3]:
books_metadata = pd.read_csv('https://raw.githubusercontent.com/AmolMavuduru/SurprisePythonExamples/master/data/books.csv.zip')
books_metadata.head(10)

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,...,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,...,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,...,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [15]:
books_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

### Creating a Dataset for Surprise

In [6]:
#Build the SVD based Collaborative filter
from surprise import SVD, Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(ratings_data[['user_id', 'book_id', 'rating']], reader)


### Cross-Validating a Simple SVD Model and train
We can train and cross-validate a model that performs SVD (singular value decomposition) in order to build a recommendation system in just a few lines of code. SVD is a popular matrix factorization algorithm that can be used for recommender systems.

The matrix factorization algorithm uses a procedure such as gradient descent to minimize the error when predicting existing ratings using the matrix factors. Thus, an algorithm like SVD builds a recommendation system by allowing us to “fill in the gaps” in the rating matrix, predicting the ratings that each user would assign to each item in the dataset.

In [7]:
from surprise.model_selection import cross_validate

svd = SVD(verbose=True, n_epochs=10)
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.8551  0.8570  0.8566  0.8562  0.0008  
MAE (testset)     0.6743  0.6758  0.6758  0.6753  0.0007  
Fit time          128.34  127.90  120.68  125.64  3.51    
Test time         20.36   21.16   23.81   21.78   1.47    


{'test_rmse': array([0.85510036, 0.85695213, 0.85663755]),
 'test_mae': array([0.67433001, 0.67577706, 0.67578979]),
 'fit_time': (128.33679103851318, 127.89906358718872, 120.67652225494385),
 'test_time': (20.358433961868286, 21.162941694259644, 23.807305574417114)}

In [8]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


### Generating Rating Predictions

Now that we have a trained SVD model, we can use it to predict the rating a user would assign to a book given an ID for the user (UID) and an ID for the item/book (IID). The code below demonstrates how to do this with the predict method.

The predict method returns the Prediction shown below, which contains a field called est that indicates the estimated book rating for this specific user.

In [12]:
svd.predict(uid=1, iid=1029)

Prediction(uid=1, iid=1029, r_ui=None, est=3.0751100574940824, details={'was_impossible': False})

### Generating Book Recommendations(Nearest title)

In [40]:
import difflib
#difflib.get_close_matches :Return a list of the best “good enough” matches. word is a sequence 
#for which close matches are desired (typically a string), and possibilities is a list of 
#sequences against which to match word (typically a list of strings).


import random

def get_book_id(book_title, metadata):
    
    existing_titles = list(metadata['title'].values)
   
    closest_titles = difflib.get_close_matches(book_title, existing_titles)
   
    book_id = metadata[metadata['title'] == closest_titles[0]]['id'].values[0]
    print (book_id,closest_titles)
    return book_id

def get_book_info(book_id, metadata):
    
    book_info = metadata[metadata['id'] == book_id][['id', 'isbn', 
                                                    'authors', 'title', 'original_title']]
    return book_info.to_dict(orient='records')

def predict_review(user_id, book_title, model, metadata):
    
    book_id = get_book_id(book_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=book_id)
    
    return review_prediction.est,book_id

def generate_recommendation(user_id, model, metadata, thresh=4):
    
    book_titles = list(metadata['title'].values)
    random.shuffle(book_titles)
    
    for book_title in book_titles:
        rating, ibook_id = predict_review(user_id, book_title, model, metadata)
        if rating >= thresh:
            #book_id = get_book_id(book_title, metadata)
            return get_book_info(ibook_id, metadata)

In [48]:
generate_recommendation(314, svd, books_metadata)# userid=1000, model=svd,metadata= books_metdata dataset

5105 ['The Geography of You and Me', 'The Catastrophic History of You and Me', 'The Last Summer (of You and Me)']
6168 ['Blue Nights', 'Blue Highways', 'White Nights']
6349 ['Temple']
267 ['The Nightingale', 'The Night Sister', 'The Night Strangers']


[{'id': 267,
  'isbn': '312577222',
  'authors': 'Kristin Hannah',
  'title': 'The Nightingale',
  'original_title': 'The Nightingale'}]

In [39]:
import difflib
 
# define original text
# taken from: https://en.wikipedia.org/wiki/Internet_Information_Services
original = ["About the IIS", "", "IIS 8.5 has several improvements related", "to performance in large-scale scenarios, such", "as those used by commercial hosting providers and Microsoft's", "own cloud offerings."]

# define modified text
edited = ["About the IIS", "", "It has several improvements related", "to performance in large-scale scenarios."]

# initiate the Differ object
d = difflib.Differ()
 
# calculate the difference between the two texts
diff = d.compare(original, edited)
 
# output the result
print ('\n'.join(diff))

  About the IIS
  
- IIS 8.5 has several improvements related
?  ^^^^^^

+ It has several improvements related
?  ^

- to performance in large-scale scenarios, such
?                                        ^^^^^^

+ to performance in large-scale scenarios.
?                                        ^

- as those used by commercial hosting providers and Microsoft's
- own cloud offerings.


In [ ]:
https://docs.python.org/3/library/difflib.html